In [27]:
class Graph:
    def __init__(self, vertices):
        self.vertices = vertices  # List of vertex labels
        self.adj_list = {vertex: [] for vertex in vertices}
        self.edges = []  # List to store all edges as tuples (u, v, cost)

    def add_edge(self, u, v, cost):
        self.adj_list[u].append((v, cost))
        self.adj_list[v].append((u, cost))  # Assuming the graph is undirected
        self.edges.append((u, v, cost))  # Store the edge for Kruskal's algorithm

    def num_vertices(self):
        return len(self.vertices)

    def vertices_reachable_from(self, u):
        return self.adj_list[u]


# Create a graph with 7 vertices labeled A to G
vertices = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
graph1 = Graph(vertices)

# Add edges to the graph
graph1.add_edge('A', 'B', 1)
graph1.add_edge('A', 'E', 1)
graph1.add_edge('A', 'D', 1)
graph1.add_edge('E', 'F', 1)
graph1.add_edge('D', 'F', 1)
graph1.add_edge('D', 'C', 1)
graph1.add_edge('F', 'C', 1)
graph1.add_edge('B', 'G', 1)
graph1.add_edge('G', 'C', 1)
graph1.add_edge('G', 'D', 1)


from queue import Queue  # Used for queue

# Function to perform Breadth-First Search (BFS) on a graph
def breadth_first_search(graph, start_node):
    visited_order = []  # To store the order of visited nodes
    visited = {vertex: False for vertex in graph.vertices}  # Tracks visited nodes
    path = {vertex: None for vertex in graph.vertices}  # Tracks the shortest path tree
    q = Queue()  # Queue for BFS

    q.put(start_node)
    visited[start_node] = True
    visited_order.append(start_node)

    while not q.empty():
        u = q.get()
        for adj_vertex, _ in graph.vertices_reachable_from(u):  # Adjusted for tuples
            if not visited[adj_vertex]:
                visited[adj_vertex] = True
                visited_order.append(adj_vertex)
                path[adj_vertex] = u
                q.put(adj_vertex)

    return visited_order, path


visited_order, path = breadth_first_search(graph1, start_node='A')

# Print the results
print("BFS Visited Order:", visited_order)
print("Path Tree:", path)

def depth_first_search(graph, start_node):
    visited_order = []
    visited = {vertex: False for vertex in graph.vertices}
    path = {vertex: None for vertex in graph.vertices}
    stack = []

    stack.append(start_node)

    while len(stack) > 0:
        u = stack.pop()

        if not visited[u]:
            visited[u] = True
            visited_order.append(u)

            for adj_vertex, _ in graph.vertices_reachable_from(u):  # Adjusted for tuples
                if not visited[adj_vertex]:
                    path[adj_vertex] = u
                    stack.append(adj_vertex)

    return visited_order, path


visited_order, path = depth_first_search(graph1, start_node='A')

# Print the results
print("DFS Visited Order:", visited_order)
print("Path Tree:", path)

BFS Visited Order: ['A', 'B', 'E', 'D', 'G', 'F', 'C']
Path Tree: {'A': None, 'B': 'A', 'C': 'D', 'D': 'A', 'E': 'A', 'F': 'E', 'G': 'B'}
DFS Visited Order: ['A', 'D', 'G', 'C', 'F', 'E', 'B']
Path Tree: {'A': None, 'B': 'G', 'C': 'G', 'D': 'A', 'E': 'F', 'F': 'C', 'G': 'D'}


In [31]:
def initialize_union_find(vertices):
    # Each vertex is its own parent initially
    return {vertex: vertex for vertex in vertices}

def find(disjoint_set, vertex):
    # Path compression
    if disjoint_set[vertex] != vertex:
        disjoint_set[vertex] = find(disjoint_set, disjoint_set[vertex])
    return disjoint_set[vertex]

def union(disjoint_set, u, v):
    # Union by rank not necessary for simplicity here
    root_u = find(disjoint_set, u)
    root_v = find(disjoint_set, v)
    if root_u != root_v:
        disjoint_set[root_u] = root_v


def kruskal_mst(graph):
    # Step 1: Sort edges by increasing cost
    sorted_edges = sorted(graph.edges, key=lambda edge: edge[2])  # Sort by cost

    # Step 2: Initialize a disjoint-set (Union-Find) structure for all vertices
    disjoint_set = initialize_union_find(graph.vertices)

    # Step 3: Initialize an empty list to store edges of the MST
    min_spanning_tree = []

    # Step 4: Process edges in sorted order
    for edge in sorted_edges:
        u, v, cost = edge

        # Step 5: Check if adding this edge forms a cycle
        if find(disjoint_set, u) != find(disjoint_set, v):
            # Step 6: Add edge to the MST
            min_spanning_tree.append(edge)

            # Step 7: Union the sets of u and v
            union(disjoint_set, u, v)

    # Step 8: Return the edges in the MST
    return min_spanning_tree


# Create a graph with 7 vertices labeled A to G
vertices = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
graph = Graph(vertices)

# Add weighted edges to the graph
graph.add_edge('A', 'B', 3)
graph.add_edge('A', 'C', 1)
graph.add_edge('B', 'D', 5)
graph.add_edge('C', 'D', 4)
graph.add_edge('C', 'E', 2)

# Find the Minimum Spanning Tree
mst = kruskal_mst(graph)

# Print the result
print("Kruskal's Minimum Spanning Tree:")
for edge in mst:
    print(edge)

import heapq  # For priority queue

def prim(graph):
    # Initialize the MST and visited set
    min_spanning_tree = []  # To store the edges in the MST
    visited = set()  # Tracks visited vertices
    min_heap = []  # Min-heap to store edges as (cost, u, v)

    # Start from the first vertex
    start_vertex = graph.vertices[0]
    visited.add(start_vertex)

    # Add all edges from the start vertex to the heap
    for neighbor, cost in graph.vertices_reachable_from(start_vertex):
        heapq.heappush(min_heap, (cost, start_vertex, neighbor))

    # Process the heap until the MST contains all vertices
    while len(min_spanning_tree) < graph.num_vertices() - 1 and min_heap:
        cost, u, v = heapq.heappop(min_heap)

        # If the target vertex is already visited, skip this edge
        if v in visited:
            continue

        # Add the edge to the MST
        min_spanning_tree.append((u, v, cost))
        visited.add(v)

        # Add all edges from the newly added vertex to the heap
        for neighbor, cost in graph.vertices_reachable_from(v):
            if neighbor not in visited:
                heapq.heappush(min_heap, (cost, v, neighbor))

    return min_spanning_tree


# Example Usage
vertices = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
graph = Graph(vertices)

# Add weighted edges to the graph
graph.add_edge('A', 'B', 3)
graph.add_edge('A', 'C', 1)
graph.add_edge('B', 'D', 5)
graph.add_edge('C', 'D', 4)
graph.add_edge('C', 'E', 2)

# Find the Minimum Spanning Tree using Prim's algorithm
mst = prim(graph)

# Print the result
print("Prim's Minimum Spanning Tree:")
for edge in mst:
    print(edge)

from queue import Queue  # Used for queue

def topological_sort(graph):
    # Initialize indegree for each vertex
    indegree = {vertex: 0 for vertex in graph.vertices}

    # Calculate indegree of each vertex
    for u in graph.vertices:
        for v, _ in graph.adj_list[u]:  # Only care about the adjacency list
            indegree[v] += 1

    # Initialize the queue with vertices having indegree 0
    q = Queue()
    for vertex, degree in indegree.items():
        if degree == 0:
            q.put(vertex)

    # Perform topological sort
    sort_result = []
    while not q.empty():
        u = q.get()
        sort_result.append(u)

        # Reduce indegree of neighbors
        for v, _ in graph.adj_list[u]:
            indegree[v] -= 1
            if indegree[v] == 0:
                q.put(v)

    # If the sort result does not contain all vertices, there is a cycle
    if len(sort_result) != graph.num_vertices():
        return None  # The graph has a cycle
    else:
        return sort_result


# Example Usage
# Create a graph with vertices labeled A to F
vertices = ['A', 'B', 'C', 'D', 'E', 'F']
graph_no_cycle = Graph(vertices)

# Add directed edges to form a DAG (no cycles)
graph_no_cycle.add_edge('A', 'B', 1)
graph_no_cycle.add_edge('A', 'C', 1)
graph_no_cycle.add_edge('B', 'D', 1)
graph_no_cycle.add_edge('C', 'D', 1)
graph_no_cycle.add_edge('D', 'E', 1)
graph_no_cycle.add_edge('E', 'F', 1)

# Perform topological sort
top_sort_result_no_cycle = topological_sort(graph_no_cycle)

# Print the result
if top_sort_result_no_cycle:
    print("Topological Sort Order:", top_sort_result_no_cycle)
else:
    print("The graph contains a cycle and cannot be topologically sorted.")


Kruskal's Minimum Spanning Tree:
('A', 'C', 1)
('C', 'E', 2)
('A', 'B', 3)
('C', 'D', 4)
Prim's Minimum Spanning Tree:
('A', 'C', 1)
('C', 'E', 2)
('A', 'B', 3)
('C', 'D', 4)
The graph contains a cycle and cannot be topologically sorted.


In [34]:
class Graph:
    def __init__(self, vertices):
        self.vertices = vertices
        self.edges = []  # For Kruskal
        self.adj_list = {vertex: [] for vertex in vertices}  # For Prim and Topological Sort

    def add_edge(self, u, v, cost):
        self.edges.append((u, v, cost))  # Add edge for Kruskal
        self.adj_list[u].append((v, cost))  # Add edge for Prim and Topological Sort

    def vertices_reachable_from(self, vertex):
        return self.adj_list[vertex]

    def num_vertices(self):
        return len(self.vertices)

# Example Usage
vertices = ['A', 'B', 'C', 'D', 'E']
graph = Graph(vertices)

# Add directed edges
graph.add_edge('A', 'B', 3)
graph.add_edge('A', 'C', 1)
graph.add_edge('B', 'D', 5)
graph.add_edge('C', 'D', 4)
graph.add_edge('C', 'E', 2)

# Kruskal's MST
print("\nKruskal's Minimum Spanning Tree:")
kruskal_result = kruskal_mst(graph)
for edge in kruskal_result:
    print(edge)

# Prim's MST
print("\nPrim's Minimum Spanning Tree:")
prim_result = prim(graph)
for edge in prim_result:
    print(edge)

# Topological Sort
print("\nTopological Sort Order:")
topological_result = topological_sort(graph)
if topological_result:
    print(topological_result)
else:
    print("The graph contains a cycle and cannot be topologically sorted.")



Kruskal's Minimum Spanning Tree:
('A', 'C', 1)
('C', 'E', 2)
('A', 'B', 3)
('C', 'D', 4)

Prim's Minimum Spanning Tree:
('A', 'C', 1)
('C', 'E', 2)
('A', 'B', 3)
('C', 'D', 4)

Topological Sort Order:
['A', 'B', 'C', 'D', 'E']
